# Showcase some of the features of skorch

This notebook introduces you to some of the nice features offered by [skorch](https://github.com/skorch-dev/skorch)

It is a companion notebook the PyCon/PyData Berlin 2019 presentation that can be found [here](https://github.com/BenjaminBossan/public-presentations/blob/master/20191010-pycon-pydata/presentation.org).

## Basic setup

### Imports

In [1]:
import numpy as np
from sklearn.datasets import make_classification
import torch
from torch import nn
import torch.nn.functional as F

### Seeds and constants

In [2]:
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0);

In [3]:
DEVICE = 'cuda'  # choose 'cuda' or 'cpu'

### A toy binary classification task

In [4]:
X, y = make_classification(10000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)

In [5]:
X.shape, y.shape, y.mean()

((10000, 20), (10000,), 0.5003)

### Definition of the PyTorch `module`

We define a vanilla neural network with one hidden layer. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [6]:
class MyModule(nn.Module):
    def __init__(self, num_units=10, dropout=0.5):
        super().__init__()

        self.dense = nn.Linear(20, num_units)
        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(num_units, 2)

    def forward(self, X, **kwargs):
        X = F.relu(self.dense(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

## Reduction of boilerplate code

### Pure PyTorch implementation

Below we show a basic training loop implemented with just PyTorch.

In [7]:
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [9]:
ds_train = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
loader_train = DataLoader(ds_train, batch_size=256, shuffle=True)
ds_valid = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))
loader_valid = DataLoader(ds_valid, batch_size=256)
module = MyModule().to(DEVICE)
optimizer = torch.optim.SGD(module.parameters(), lr=0.02)
criterion = nn.NLLLoss()
template = "epoch: {} | loss train: {:.4f} | loss valid: {:.4f} | acc valid: {:.4f} | dur: {:.3f}"

In [10]:
for epoch in range(20):
    tic = time.time()
    losses_train = []
    for Xb, yb in loader_train:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        y_proba = module(Xb)
        loss = criterion(torch.log(y_proba), yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses_train.append(loss.item())
        
    losses_valid = []
    accuracy_valid = []
    for Xb, yb in loader_valid:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        y_proba = module(Xb)
        loss = criterion(torch.log(y_proba), yb)
        optimizer.step()
        optimizer.zero_grad()
        losses_valid.append(loss.item())
        accuracy_valid.append(accuracy_score(yb.cpu().numpy(), y_proba.argmax(1).cpu().numpy()))
        
    toc = time.time() - tic
    print(template.format(
        epoch + 1, np.mean(losses_train), np.mean(losses_valid), np.mean(accuracy_valid), toc))

epoch: 1 | loss train: 0.6813 | loss valid: 0.6524 | acc valid: 0.5941 | dur: 0.226
epoch: 2 | loss train: 0.6420 | loss valid: 0.6199 | acc valid: 0.6419 | dur: 0.116
epoch: 3 | loss train: 0.6305 | loss valid: 0.6081 | acc valid: 0.6516 | dur: 0.089
epoch: 4 | loss train: 0.6206 | loss valid: 0.6026 | acc valid: 0.6636 | dur: 0.090
epoch: 5 | loss train: 0.6006 | loss valid: 0.5945 | acc valid: 0.6597 | dur: 0.084
epoch: 6 | loss train: 0.5867 | loss valid: 0.5763 | acc valid: 0.6899 | dur: 0.089
epoch: 7 | loss train: 0.5776 | loss valid: 0.5593 | acc valid: 0.6984 | dur: 0.084
epoch: 8 | loss train: 0.5628 | loss valid: 0.5546 | acc valid: 0.7001 | dur: 0.090
epoch: 9 | loss train: 0.5463 | loss valid: 0.5418 | acc valid: 0.7175 | dur: 0.091
epoch: 10 | loss train: 0.5377 | loss valid: 0.5287 | acc valid: 0.7244 | dur: 0.089
epoch: 11 | loss train: 0.5308 | loss valid: 0.5211 | acc valid: 0.7345 | dur: 0.090
epoch: 12 | loss train: 0.5181 | loss valid: 0.5121 | acc valid: 0.7427 | 

### Training with skorch

Now we show how to achieve the same outcome with skorch. Note how we don't need to make any adjustments to the `module`.

In [11]:
from skorch import NeuralNetClassifier

In [12]:
net = NeuralNetClassifier(
    MyModule,
    module__num_units=50,
    max_epochs=20,
    lr=0.02,
    batch_size=256,
    iterator_train__shuffle=True,
    device=DEVICE,
)

In [13]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6981       0.6992        0.6096  0.1066
      2        0.6155       0.7441        0.5650  0.1433
      3        0.5799       0.7671        0.5319  0.1370
      4        0.5485       0.7871        0.5050  0.1291
      5        0.5321       0.8001        0.4833  0.1281
      6        0.5163       0.8101        0.4649  0.1103
      7        0.4959       0.8226        0.4465  0.1231
      8        0.4869       0.8341        0.4312  0.1400
      9        0.4721       0.8421        0.4180  0.1423
     10        0.4663       0.8481        0.4052  0.1434
     11        0.4455       0.8541        0.3937  0.1077
     12        0.4395       0.8581        0.3826  0.1210
     13        0.4311       0.8616        0.3726  0.1277
     14        0.4250       0.8671        0.3641  0.1078
     15        0.4176       0.8696        0.3569  0.1387
     16        0.4095       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
)

## Compatibility with sklearn API

### Monitor sklearn metrics during training

In [14]:
from skorch.callbacks import EpochScoring
from sklearn.metrics import roc_auc_score

In [15]:
auc = EpochScoring(
    scoring=roc_auc_score,  # <-- just passing 'roc_auc' would also work
    lower_is_better=False,
)

In [16]:
net = NeuralNetClassifier(
    MyModule,
    module__num_units=50,
    max_epochs=20,
    lr=0.02,
    batch_size=256,
    iterator_train__shuffle=True,
    device=DEVICE,
    callbacks=[auc],
)

In [17]:
net.fit(X, y)

  epoch    roc_auc_score    train_loss    valid_acc    valid_loss     dur
-------  ---------------  ------------  -----------  ------------  ------
      1           0.6527        0.7091       0.6527        0.6307  0.1310
      2           0.7216        0.6320       0.7216        0.5851  0.1100
      3           0.7571        0.5908       0.7571        0.5520  0.1362
      4           0.7741        0.5670       0.7741        0.5248  0.1348
      5           0.7896        0.5467       0.7896        0.5016  0.1266
      6           0.8086        0.5273       0.8086        0.4800  0.1219
      7           0.8161        0.5090       0.8161        0.4614  0.1170
      8           0.8221        0.4929       0.8221        0.4450  0.1462
      9           0.8296        0.4874       0.8296        0.4305  0.1189
     10           0.8371        0.4708       0.8371        0.4160  0.1183
     11           0.8451        0.4642       0.8451        0.4035  0.1219
     12           0.8486        0.4489

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
)

### Support for the basic methods

In [18]:
from sklearn.base import clone
from sklearn.model_selection import cross_validate

In [19]:
y_pred = net.predict(X[:5])
y_pred

array([0, 1, 1, 0, 1])

In [20]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[0.75151503, 0.24848492],
       [0.15232019, 0.8476798 ],
       [0.18867724, 0.81132275],
       [0.5423922 , 0.45760784],
       [0.40528426, 0.5947157 ]], dtype=float32)

In [21]:
net.get_params();

In [22]:
net.set_params(verbose=0)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
)

In [23]:
_ = clone(net)

In [24]:
cross_validate(net, X, y, cv=3)

/home/vinh/anaconda3/envs/skorch/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([1.9804399 , 1.77662516, 1.96318698]),
 'score_time': array([0.03349638, 0.03385663, 0.03398347]),
 'test_score': array([0.86502699, 0.87822436, 0.8772509 ]),
 'train_score': array([0.86843684, 0.87983798, 0.87177564])}

### Use inside an sklearn `Pipeline`

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [26]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('net', <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
))])

In [27]:
pipe.predict(X[:5])

array([0, 1, 1, 0, 1])

In [28]:
pipe.predict_proba(X[:5])

array([[0.50151825, 0.49848178],
       [0.25543442, 0.7445656 ],
       [0.37828928, 0.62171066],
       [0.5110312 , 0.48896876],
       [0.40677232, 0.5932276 ]], dtype=float32)

### Pickle the whole pipeline

In [29]:
import pickle

Saves the whole pipeline, including preprocessing and the neural net.

In [30]:
with open('my_pipeline.pickle', 'wb') as f:
    pickle.dump(pipe, f)

/home/vinh/anaconda3/envs/skorch/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MyModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### GridSearchCV

No special adjustments need to be made to perform a hyperparameter search on the net parameters. We can even search on the `__init__` parameters of our `module` by using the `'module__'` prefix.

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
params = {
    'max_epochs': [10, 20],
    'optimizer__momentum': [0.0, 0.9],
    'module__num_units': [10, 50],  # <-- just works
    'module__dropout': [0, 0.5],  # <-- just works
}

In [33]:
%time search = GridSearchCV(net, params, verbose=2, cv=3).fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0, total=   1.1s
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0, total=   0.9s
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0 
[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.0, total=   1.0s
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9 
[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9, total=   1.0s
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9 
[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9, total=   1.1s
[CV] max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9 
[CV]  max_epochs=10, module__dropout=0, module__num_units=10, optimizer__momentum=0.9, total=   1.0s
[CV] max_epochs=10, module__dropout=0, module__num_units=50, optimizer__momentum=0.0 
[CV]  max_epochs=10, module__dropout=0, module__num_units=50, opt

[CV]  max_epochs=20, module__dropout=0.5, module__num_units=50, optimizer__momentum=0.9, total=   2.2s
[CV] max_epochs=20, module__dropout=0.5, module__num_units=50, optimizer__momentum=0.9 
[CV]  max_epochs=20, module__dropout=0.5, module__num_units=50, optimizer__momentum=0.9, total=   2.0s
[CV] max_epochs=20, module__dropout=0.5, module__num_units=50, optimizer__momentum=0.9 
[CV]  max_epochs=20, module__dropout=0.5, module__num_units=50, optimizer__momentum=0.9, total=   2.2s


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  1.3min finished


CPU times: user 5min 9s, sys: 16.7 s, total: 5min 26s
Wall time: 1min 21s


In [34]:
search.best_score_, search.best_params_

(0.9565,
 {'max_epochs': 20,
  'module__dropout': 0,
  'module__num_units': 50,
  'optimizer__momentum': 0.9})

#### Grid search everything!

You can grid search the parameters of almost everything:

- NeuralNet
- module
- optimizer
- criterion
- DataLoader
- callbacks

Just use the `__` notation known from sklearn, e.g. `optimizer__momentum` to set the `momentum` parameter of the optimizer. To make a search on callback parameters, give the parameter a name and use that name to dispatch to the callback. E.g.:

```
net = NeuralNetClassifier(..., callbacks=[('mycb', MyCallback(foo=1))])
params = {'callbacks__mycb__foo': [1, 2, 3]}
```

### Swap skorch net for any other sklearn estimator

Since skorch's estimators work like any other sklearn estimator, you can swap them out to see which one leads to the best results.

Here we compare our neural network with a logistic regression and a KNN classifier.

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [36]:
net.set_params(**search.best_params_)  # use the best parameters from grid search

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
)

In [37]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('model', net),
])
params = {'model': [net, LogisticRegression(), KNeighborsClassifier()]}
search = GridSearchCV(pipe, params, verbose=2, cv=3)

In [38]:
%time search.fit(X, y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
), total=   2.5s
[CV] model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV]  model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
), total=   2.5s
[CV] model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
) 
[CV]  model=<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
), total=   2.1s
[CV] model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_st

/home/vinh/anaconda3/envs/skorch/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/vinh/anaconda3/envs/skorch/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/vinh/anaconda3/envs/skorch/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[CV]  model=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'), total=   0.8s
[CV] model=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform') 
[CV]  model=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'), total=   0.8s
[CV] model=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform') 
[CV]  model=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'), total=   0.8s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.1s finished


CPU times: user 46.3 s, sys: 2.06 s, total: 48.3 s
Wall time: 16.9 s


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'model': [<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
), LogisticRegression(C=1.0, class_...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [39]:
search.best_score_, search.best_params_

(0.9491,
 {'model': <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
    module_=MyModule(
      (dense): Linear(in_features=20, out_features=50, bias=True)
      (dropout): Dropout(p=0)
      (output): Linear(in_features=50, out_features=2, bias=True)
    ),
  )})

### distributed `GridSearchCV` with dask

To run a distributed hyperparameter search, you need `dask` and `dask.distributed`:

`$ pip install dask distributed`

Setup your dask workers as described [here](https://docs.dask.org/en/latest/setup.html).

Then run the following lines:

```
from dask.distributed import Client
from joblib import parallel_backend

client = Client('127.0.0.1:8786')

search = GridSearchCV(net, params, verbose=2, cv=3)

with parallel_backend('dask'):
    search.fit(X, y)
```

## More additions

### Save the `state_dict`

If we just want to save the `state_dict` of our module (and maybe our optimizer), we can either use the `Checkpoint` callback or call the `save_params` method. Use `load_params` to load the `state_dict` later on.

In [40]:
from skorch.callbacks import Checkpoint

In [41]:
cp = Checkpoint(monitor='valid_loss_best', dirname='exp1')
net = NeuralNetClassifier(
    MyModule,
    module__num_units=50,
    max_epochs=20,
    lr=0.02,
    batch_size=256,
    iterator_train__shuffle=True,
    device=DEVICE,
    callbacks=[cp],
)

In [42]:
net.fit(X, y)  # Checkpoint saves each time valid lost improves

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.6848       0.6957        0.5929     +  0.1037
      2        0.5999       0.7401        0.5464     +  0.0993
      3        0.5626       0.7686        0.5148     +  0.0991
      4        0.5356       0.7851        0.4899     +  0.0962
      5        0.5129       0.8006        0.4678     +  0.0962
      6        0.4924       0.8126        0.4470     +  0.0970
      7        0.4868       0.8216        0.4311     +  0.1022
      8        0.4708       0.8281        0.4149     +  0.1355
      9        0.4513       0.8396        0.3999     +  0.1009
     10        0.4426       0.8486        0.3868     +  0.0993
     11        0.4368       0.8596        0.3752     +  0.0987
     12        0.4346       0.8631        0.3654     +  0.0991
     13        0.4226       0.8696        0.3554     +  0.1096
     14        0.4116       0.8736        0.3465     + 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=50, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=50, out_features=2, bias=True)
  ),
)

In [43]:
net.save_params(
    f_params='exp1/mynet.pt',  # <- state dict of module
    f_optimizer='exp1/myoptimizer.pt',  # <- state dict of optimizer
)

### Handling of different data formats

By default, skorch handles the most common data formats, even more complex ones like dictionaries. If this doesn't fit your need, just define your own `Dataset`.

- numpy arrays
- PyTorch Datasets (most)
- dict or list of arrays
- pandas DataFrames
- scipy sparse CSR matrices

### Callbacks

skorch comes packaged with a few useful callbacks:

In [44]:
from skorch.callbacks import GradientNormClipping
from skorch.callbacks import LRScheduler
from skorch.callbacks import EpochScoring, BatchScoring
from skorch.callbacks import Checkpoint, TrainEndCheckpoint, LoadInitState
from skorch.callbacks import Freezer
from skorch.callbacks import TensorBoard

### CLI

With the help of skorch and Google's fire library, it is exceedingly easy to transform your training script into a nice CLI. This is what skorch and fire will automatically take care of:

* help for the CLI usage
* show docstrings in CLI help
* set __all__ possible parameters from the command line without any manuel argument parsing

First install fire and numpydoc:
    
`$ pip install fire numpydoc`

It requires only a few lines of code to turn your script into a nice CLI:

```
def main(..., **kwargs):
    model = ...  # put model definition here

    model = parse_args(kwargs)(model)  # <-- add this line
    
    model.fit(X, y)


if __name__ == '__main__':
    fire.Fire(main)
```

Here is the complete train.py script. Note the few lines that needed to be added:

In [45]:
!cat train.py

"""Simple training script for a MLP classifier.

See accompanying `pycon_showcase_skorch.ipynb` for details.

"""

import pickle

import fire
import numpy as np
from sklearn.datasets import make_classification
from skorch import NeuralNetClassifier
import torch
from torch import nn

from skorch.helper import parse_args


np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)


# number of input features
N_FEATURES = 20

# number of classes
N_CLASSES = 2

# custom defaults for net
DEFAULTS = {
    'batch_size': 256,
    'module__hidden_units': 30,
}


class MLPClassifier(nn.Module):
    """A simple multi-layer perceptron module.

    This can be adapted for usage in different contexts, e.g. binary
    and multi-class classification, regression, etc.

    Note: This docstring is used to create the help for the CLI.

    Parameters
    ----------
    hidden_units : int (default=10)
      Number of units in hidden layers.

    num_hidden : int (default=1)
      Number of hidden l

General help:

In [46]:
!python train.py -- --help

NAME
    train.py - Train an MLP classifier on synthetic data.

SYNOPSIS
    train.py <flags>

DESCRIPTION
    n_samples : int (default=100)
      Number of training samples

    output_file : str (default=None)
      If not None, file name used to save the model.

    kwargs : dict
      Additional model parameters.

FLAGS
    --n_samples=N_SAMPLES
    --output_file=OUTPUT_FILE
    Additional flags are accepted.


Model-specific help:

In [47]:
!python train.py --help

This is the help for the model-specific parameters.
To invoke help for the remaining options, run:
python train.py -- --help

<NeuralNetClassifier> options:
  --module : torch module (class or instance)
    A PyTorch :class:`~torch.nn.Module`. In general, the
    uninstantiated class should be passed, although instantiated
    modules will also work.
  --criterion : torch criterion (class, default=torch.nn.NLLLoss)
    Negative log likelihood loss. Note that the module should return
    probabilities, the log is applied during ``get_loss``.
  --optimizer : torch optim (class, default=torch.optim.SGD)
    The uninitialized optimizer (update rule) used to optimize the
    module
  --lr : float (default=0.01)
    Learning rate passed to the optimizer. You may use ``lr`` instead
    of using ``optimizer__lr``, which would result in the same outcome.
  --max_epochs : int (default=10)
    The number of epochs to train for each ``fit`` call. Note that you
    may keyboard-interrupt training a

This is how you can call the script from the command line:

In [48]:
!python train.py --n_samples 1000 --output_file 'exp1/model.pkl' --lr 0.1 --max_epochs 5 \
  --device 'cuda' --module__hidden_units 50 --module__nonlin 'torch.nn.RReLU(0.1, upper=0.4)'\
  --callbacks__valid_acc__on_train --callbacks__valid_acc__name train_acc

Training MLP classifier
  epoch    train_acc    train_loss    valid_loss     dur
-------  -----------  ------------  ------------  ------
      1       0.7772        0.5813        0.5013  0.1235
      2       0.9024        0.4874        0.4314  0.0175
      3       0.9224        0.4195        0.3786  0.0165
      4       0.9274        0.3670        0.3388  0.0149
      5       0.9324        0.3259        0.3079  0.0108
Saved model to file 'exp1/model.pkl'.


Note how you can even pass Python objects as arguments like `--module__nonlin 'torch.nn.RReLU(0.1, upper=0.4)'`.

## Easily hackable

We made sure that skorch is as hackable as possible. On the neural net classes, look out for methods that start with `get_`, such as `get_loss`, or override the `train_step` itself. On the callbacks, look for methods that start with `on_`, such as `on_train_begin`. They always receive the associated `net` instance as the first parameter.

### Custom callbacks

In [49]:
from skorch.callbacks import Callback

In [50]:
def send_tweet(msg):
    print("*tweet* {}".format(msg))


class TweetAccuracy(Callback):
    def __init__(self, min_accuracy=0.99):
        self.min_accuracy = min_accuracy

    def on_train_end(self, net, **kwargs):
        best_accuracy = max(net.history[:, 'valid_acc'])
        if best_accuracy >= self.min_accuracy:
            msg = "Reached an accuracy of {:.4f}!!!".format(best_accuracy)
            send_tweet(msg)

### Implement gradient accumulation

In [51]:
class GradAccNet(NeuralNetClassifier):
    def __init__(self, *args, acc_steps=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.acc_steps = acc_steps

    def get_loss(self, *args, **kwargs):
        loss = super().get_loss(*args, **kwargs)
        return loss / self.acc_steps  # normalize loss

    def train_step(self, Xi, yi, **fit_params):
        n_train_batches = len(self.history[-1, 'batches'])
        step = self.train_step_single(Xi, yi, **fit_params)

        if n_train_batches % self.acc_steps == 0:
            self.optimizer_.step()
            self.optimizer_.zero_grad()
        return step

#### Putting it together

In [52]:
grad_acc_net = GradAccNet(MyModule, callbacks=[TweetAccuracy(min_accuracy=0.7)])

In [53]:
grad_acc_net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.3835       0.5007        0.3582  0.1253
      2        0.3625       0.5172        0.3446  0.1314
      3        0.3501       0.5622        0.3361  0.1157
      4        0.3399       0.6052        0.3296  0.1082
      5        0.3352       0.6312        0.3240  0.1283
      6        0.3287       0.6637        0.3184  0.1125
      7        0.3256       0.6932        0.3133  0.1257
      8        0.3209       0.7041        0.3082  0.1100
      9        0.3149       0.7186        0.3028  0.1106
     10        0.3113       0.7286        0.2978  0.1108
*tweet* Reached an accuracy of 0.7286!!!


<class '__main__.GradAccNet'>[initialized](
  module_=MyModule(
    (dense): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)